In [60]:
from sentence_transformers import SentenceTransformer
import requests 
import numpy as np
from tqdm import tqdm
import pandas as pd
from elasticsearch import Elasticsearch

# Q1

In [4]:
model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.52k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

C:\Users\User\anaconda3\envs\llm\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [27]:
user_question = "I just discovered the course. Can I still join it?"
v = embedding_model.encode(user_question)
v[0]

0.078222625

# Prepare the documents

In [10]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [14]:
documents = [doc for doc in documents if doc['course'] == 'machine-learning-zoomcamp' ]
len(documents)

375

# Q2

In [70]:
embeddings = []

for doc in tqdm(documents):
    embedding = embedding_model.encode(f"{doc['question']} {doc['text']}")
    embeddings.append(embedding)
    doc['text_vector'] = embedding.tolist()

X = np.array(embeddings)
X.shape

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 375/375 [00:10<00:00, 34.72it/s]


(375, 768)

# Q3

In [71]:
v.dot(v)

1.0

In [72]:
scores = X.dot(v)
scores.max()

0.6506573

# Vector search

In [ ]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)

In [75]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

# Q4

In [76]:
def question_vector_knn(q):
    question = q['question']

    v_q = embedding_model.encode(question)

    return search_engine.search(v_q, num_results=5)

In [77]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [78]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [79]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [80]:
evaluate(ground_truth, question_vector_knn)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1830/1830 [00:52<00:00, 34.79it/s]


{'hit_rate': 0.9398907103825137, 'mrr': 0.8516484517304189}

# Q5

In [81]:
es_client = Elasticsearch('http://localhost:9200') 

In [82]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}
index_name = "course-questions-homework"

In [83]:
es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions-homework'})

In [84]:
for doc in tqdm(documents):
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 375/375 [00:25<00:00, 14.68it/s]


In [88]:
knn_query = {
    "field": "text_vector",
    "query_vector": v,
    "k": 5,
    "num_candidates": 10000
}

response = es_client.search(
    index=index_name,
    query={
        "match": {"section": "General course-related questions"},
    },
    knn=knn_query,
    size=5,
)

In [96]:
hits = response['hits']['hits']
max(hits, key=lambda x: x['_score'])['_source']['id']

'ee58a693'

# Q6

In [98]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [102]:
def question_vector_es_knn(q):
    question = q['question']
    course = q['course']

    v_q = embedding_model.encode(question)

    return elastic_search_knn('text_vector', v_q, course)

In [103]:
evaluate(ground_truth, question_vector_es_knn)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1830/1830 [02:57<00:00, 10.29it/s]


{'hit_rate': 0.9398907103825137, 'mrr': 0.8516484517304189}